In [4]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
% matplotlib inline

# 1. Did conversations with the new conversation page book at a higher rate?

Need to query the testsegmentation table and conversations table.


In [ ]:
#SQL Query 
'''
SELECT a.test_group, COUNT(a.test_name) AS total, COUNT(b.booked_at)
FROM conversations_conversation b
JOIN people_testsegmentation a 
ON b.requester_id = a.person_id
WHERE a.test_name = 'New Conversation Flow'
GROUP BY a.test_group;
'''

In [5]:
new_convos = pd.read_csv('new_convo.csv')
new_convos

,test_group,total,COUNT(b.booked_at)
0,holdout,22895,9174
1,variant,22941,10136


In [6]:
new_convos.rename(columns={'test_group': 'Test Group', 'total': 'Total', 'COUNT(b.booked_at)': 'Number of Bookings'}, inplace=True)
new_convos.head()

,Test Group,Total,Number of Bookings
0,holdout,22895,9174
1,variant,22941,10136


In [7]:
new_convos['Booking Rate'] = (new_convos['Number of Bookings'] / new_convos['Total']) * 100
new_convos

,Test Group,Total,Number of Bookings,Booking Rate
0,holdout,22895,9174,40.069884
1,variant,22941,10136,44.182904


It appears that the Variant group has a higher booking rate than the hold out (44.18% vs. 40.07%)

What is the gross Booking Rate for both the holdout and variant groups?

In [9]:
gross_booking_rate = (9174+10136) / (22895+22941) * 100
gross_booking_rate

42.128457980626585

# 2. Is it statistically significant?

Step 1. Set up the Null and Alternative Hypothesis


H0 = There is no difference between booking rate in the holdout and variant groups.


H1 = There IS a difference between booking rates int the holdout and variant groups.


Step 2. Set arbitrary alphas we'll use a list of [.05, .01, .001] for multiple tests.

Step 3. Need to requery the database and get all the holdout and variant data.

In [ ]:
#SQL Query 
'''
SELECT a.test_group, a.test_name, b.booked_at
FROM conversations_conversation b
JOIN people_testsegmentation a
ON b.requester_id = a.person_id
WHERE a.test_name = 'New Conversation Flow';
'''

In [10]:
stats_test = pd.read_csv('stats_test.csv')
stats_test.head()

,test_group,test_name,booked_at
0,holdout,New Conversation Flow,2017-05-26 21:02:09.499404
1,holdout,New Conversation Flow,NaN
2,variant,New Conversation Flow,2017-05-29 01:27:14.256611
3,variant,New Conversation Flow,2017-06-15 15:00:33.124108
4,variant,New Conversation Flow,2017-05-16 00:40:10.332902


Lets convert to a boolean True/False and 1/0 framework.

In [12]:
stats_test = stats_test.fillna(0)
stats_test.head()

,test_group,test_name,booked_at
0,holdout,New Conversation Flow,2017-05-26 21:02:09.499404
1,holdout,New Conversation Flow,0
2,variant,New Conversation Flow,2017-05-29 01:27:14.256611
3,variant,New Conversation Flow,2017-06-15 15:00:33.124108
4,variant,New Conversation Flow,2017-05-16 00:40:10.332902


In [29]:
stats_test.shape

(45836, 3)

In [14]:
stats_test['booked_at'] = stats_test['booked_at'].apply(lambda change = 1: change != 0)

In [15]:
stats_test.head()

,test_group,test_name,booked_at
0,holdout,New Conversation Flow,True
1,holdout,New Conversation Flow,False
2,variant,New Conversation Flow,True
3,variant,New Conversation Flow,True
4,variant,New Conversation Flow,True


Needs to be a 1/0 lets try something else. 

In [16]:
stats_test['booked_at'] = stats_test['booked_at'] * 1

In [17]:
stats_test.head()

,test_group,test_name,booked_at
0,holdout,New Conversation Flow,1
1,holdout,New Conversation Flow,0
2,variant,New Conversation Flow,1
3,variant,New Conversation Flow,1
4,variant,New Conversation Flow,1


We will be using the scipy.stats ttest_ind:

It calculates the T-test for the means of two independent samples of scores.

This is a two-sided test for the null hypothesis that 2 independent samples have identical average (expected) values. This test assumes that the populations have identical variances by default.

In [18]:
ind_1 = stats_test[stats_test['test_group'] == 'holdout']['booked_at']
ind_2 = stats_test[stats_test['test_group'] == 'variant']['booked_at']

In [19]:
ttest = stats.ttest_ind(ind_1,ind_2)
ttest 

Ttest_indResult(statistic=-8.9244437702831156, pvalue=4.6409650880993761e-19)

In [28]:
ttest_welchs = stats.ttest_ind(ind_1,ind_2,equal_var=False)
ttest_welchs

Ttest_indResult(statistic=-8.9245629485013609, pvalue=4.636000711848832e-19)

Calculated both the Welch's T-Test as well as the regular T-Test since we dont know much about the variance in the two populations, both came back very close to one another so I am making the assumption that there is not a high level of variance between the holdout and variant groups. 

### Analysis:


At all three alphas [.05,.01,001] I would reject H1 since the p-value is very small. I would not think the two booking rates are statistically significant. 

# 3. Do you have any reservations about the experiment design? What would yourecommend as next steps?

I know that everyone is selected at random but if you have a fairly new user in the holdout and a very experienced user in the variant group that could have an effect on the booking rate. 

Also with the redesign of the message page were things changed incrementally or were users presented two radically different platforms? To me it seems like you would want to augment things one at a time to determine if a certain feature on the new page led to higher booking rates. For example, if a button to view certain aspects of the users was made more/ less visible I would want to make sure that change was documented. 

Since I ran two T-Tests I would like to dig into the variation of the populations more and determine which one is best to use.


## Next Steps

I would like to implement an A/B test where the platform was changed one item at time and record booking rates to really get a feel for what features mean the most to users.

It may be useful to add in the text between the two interacting to get a feel for how the holdout/ variant site is adding or detracting from the UI.

Have individuals be part of multiple holdout/variant groups, that way we could better understand how repeat users are interacting with the site and how our changes affect the ease and ability to book.
